# UofMN Visualization and Analytics Bootcamp 
## Group 3 - Project 1 Data Collection Notebook 04/13/2019
#### James Dietz, Stephanie Hartje, Chris Howard, Emily Hudson, Marie Wothe


* All imported packages
* Note - all outputs have been commented out to avoid over-writing live project data

In [15]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from config import omdb_key

##### Collect data from bechdeltest.com

In [3]:
# Chris Howard
# Create base URL and response script for bechdeltest.com
# url = "http://bechdeltest.com/api/v1/getAllMovies"

# response = requests.get(url).json()

In [5]:
# Chris Howard
# Check initial frame of response to confirm successful data collection.
#pprint(response[0])

{'id': '8040',
 'imdbid': '0392728',
 'rating': '0',
 'title': 'Roundhay Garden Scene',
 'year': '1888'}


In [6]:
# Chris Howard
# Create dataframe from the response
bechdel_df = pd.DataFrame(response)

In [19]:
# Chris Howard
# Look at header of response and save to .csv
bechdel_df.head()
#bechdel_df.to_csv("Data/bechdel_full.csv")

In [20]:
# Chris Howard
# Data set too large to collect from OMDB API in one download, split into 9 sub-1000 entry sets
# Perform the split and test the initial save
split_df = np.array_split(bechdel_df, 9)
tue_1 = split_df[0]
#tue_1['imdbid'].to_csv('Data/james_tues.csv')

In [21]:
# Chris Howard
# Save remaining sections of data split into .csv files for each team member to use for OMDB downloads
tue_2 = split_df[1]
#tue_2['imdbid'].to_csv('Data/emily_tues.csv')
tue_3 = split_df[2]
#tue_3['imdbid'].to_csv('Data/stephanie_tues.csv')
tue_4 = split_df[3]
#tue_4['imdbid'].to_csv('Data/marie_tues.csv')
tue_5 = split_df[4]
#tue_5['imdbid'].to_csv('Data/chris_tues.csv')
wed_1 = split_df[5]
#wed_1['imdbid'].to_csv('Data/chris_wed.csv')
wed_2 = split_df[6]
#wed_2['imdbid'].to_csv('Data/marie_wed.csv')
wed_3 = split_df[7]
#wed_3['imdbid'].to_csv('Data/stephanie_wed.csv')
wed_4 = split_df[8]
#wed_4['imdbid'].to_csv('Data/emily_wed.csv')

In [24]:
# Chris Howard
# Confirm that the end of the final split is the same as the end of the original response dataframe
wed_4.tail()

,id,imdbid,rating,title,year
8071,8617,5316540,3,Close,2019
8072,8642,0437086,3,Alita: Battle Angel,2019
8073,8655,7458762,0,Le chant du loup,2019
8074,8659,2452244,1,Isn&#39;t It Romantic,2019
8075,8668,4154664,3,Captain Marvel,2019


#### Download data from OMDB API
* Note - this is just an example, each team member had a similar notebook to run on consecutive days to collect all data.

In [ ]:
# Chris Howard
# OMDB URL, confirm format with print statement
base_url = f'http://www.omdbapi.com/?apikey={omdb_key}&i=tt'
#print(base_url)

In [ ]:
# Chris Howard
# This section to be run on the first day to collect first segment of OMDB data associated with Bechdel set
# Run on Tuesday
tues_data = pd.read_csv('Data/chris_tues.csv', header=None)

omdb_ids = tues_data.iloc[:,1]


movie_data = []

for omdb_id in omdb_ids:
    # convert ID field to string
    omdb_id = str(omdb_id)
    
    # replace leading zeros for IMDB id numbers lost in .csv save
    while len(omdb_id) < 7:
        omdb_id = '0' + omdb_id
    
    # get response from OMDB and store in movie_data list
    url = base_url + omdb_id
    response = requests.get(url).json()
    movie_data.append(response)

# Create datafram from movie responses
movies = pd.DataFrame(movie_data)

# Save to csv file
#movies.to_csv('Data/chris_tues_moviedata.csv')

In [ ]:
# Chris Howard
# This section to be run on the second day to collect the second segment of OMDB data associated with Bechdel set
# Run on Wednesday
tues_data = pd.read_csv('Data/chris_wed.csv', header=None)

omdb_ids = tues_data.iloc[:,1]


movie_data = []

for omdb_id in omdb_ids:
    # convert ID field to string
    omdb_id = str(omdb_id)
    
    # replace leading zeros for IMDB id numbers lost in .csv save
    while len(omdb_id) < 7:
        omdb_id = '0' + omdb_id
    
    # get response from OMDB and store in movie_data list
    url = base_url + omdb_id
    response = requests.get(url).json()
    movie_data.append(response)

# Create datafram from movie responses
movies = pd.DataFrame(movie_data)

# Save to csv file
#movies.to_csv('Data/chris_wed_moviedata.csv')

#### Web Scraping for Revenue Data based on script from zacharyang@GitHub
* This data was collected, but ultimately not used for our analysis

In [ ]:
# Chris Howard
# Different import syntax, completed in separate notebook.
# Credit to zacharyang@GitHub for scraping script. 
import bs4
from bs4 import BeautifulSoup
import requests as rq
import re
import pandas as pd
import numpy as np

def convdollar(x):
    """
    Just a parsing function converting 2.5k to 2500, 1mil to 1000000
    """
    if 'k' in x:
        return float(x.replace('k',''))*1000
    else:
        return float(x)*1000000

def scrape():
    """
    Gets all box office data from 1989 to 2018 from boxofficemojo.com
    """
    years=[str(a) for a in range(1980,2019)]
    df_list=[]
    for year in years:
        r=rq.get('https://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr=%s&p=.htm' % year)
        print('Box Office data for %s scraped' % year)
        p=BeautifulSoup(r.text,'html.parser')
        ### Look for the table ### 
        b=p.find_all('table')
        ### Usually the fourth table object on page ### 
        tb=b[3].find_all('td')
        data=[]
        for i in tb:
            if i.find('a')!=None:
                data.append(i.find('a').contents[0])
            elif i.find('font')!=None:
                 data.append(i.find('font').contents[0])
            elif i.find('b')!=None:
                data.append(i.find('b').contents[0])
        ### Still a <b> tag left for <font> tags ## 
        data=[a.contents[0] if type(a)!=bs4.element.NavigableString else a for a in data]
        ### Strip special characters ### 
        data=[re.sub('[^A-Za-z0-9-. ]+', '', a) for a in data]
        ### Fill NaNs ### 
        data=[np.nan if a =='na' else a for a in data]
        columns=['bo_year_rank','title','studio','worldwide-gross','domestic-gross','domestic-pct','overseas-gross','overseas-pct']
        to_df=data[6:]
        if len(to_df)%len(columns) != 0:
            print('Possible table misalignment in table for year %s' % year)
            break 
        nrow=int(len(to_df)/len(columns))
        df=pd.DataFrame(np.array(to_df).reshape(nrow,8),columns=columns)
        df[['worldwide-gross','domestic-gross','overseas-gross']]=df[['worldwide-gross','domestic-gross','overseas-gross']].applymap(lambda x:convdollar(x))
        df['bo_year']=int(year)
        df_list.append(df)

    main=pd.concat(df_list)

    main.to_csv('./Data/annual_mojo1980.csv')

    
if __name__ == "__main__": 

    scrape()


#### Pull all OMDB data files together into a single file

In [ ]:
# Chris Howard
# Read each file into a dataframe and append together
data1 = pd.read_csv('chris_tues_moviedata.csv')
data2 = pd.read_csv('chris_wed_moviedata.csv')
data3 = pd.read_csv('emily_tues_moviedata.csv')
data4 = pd.read_csv('emily_wed_moviedata.csv')
data5 = pd.read_csv('james_tues_moviedata.csv')
data6 = pd.read_csv('marie_tues_moviedata.csv')
data7 = pd.read_csv('marie_wed_moviedata.csv')
data8 = pd.read_csv('stephanie_tues_moviedata.csv')
data9 = pd.read_csv('stephanie_wed_moviedata.csv')
data1 = data1.append([data2, data3, data4, data5, data6, data7, data8, data9])

#data1.to_csv('full_moviedata.csv')

#### Merge OMDB and Bechdeltest.com data into a single file

In [ ]:
# Emily Hudson
# Read in OMDB and bechdeltest.com .csv files
df = pd.read_csv("full_moviedata.csv")
df_b = pd.read_csv("bechdel_full.csv")

In [ ]:
# Emily Hudson
# Change imdbID column to type string, confirm column names for OMDB data
df["imdbID"]=df["imdbID"].astype(str)
df.columns

In [ ]:
# Emily Hudson
# Confirm column names for bechdeltest.com data
df_b.columns

In [ ]:
# Emily Hudson
# Change bechdelltest.com imdbID column to type string, then add back zeros stripped by .csv
# then append 'tt' for proper IMDB id format and save back to dataframe
df_b["imdbid"]=df_b["imdbid"].astype(str)
newID = []

for id in df_b["imdbid"]:
    while len(id) < 7:
        id = '0' + str(id) 
    id = 'tt' + str(id)
    newID.append(id)
df_b["imdbID"] = newID

In [ ]:
# Emily Hudson
# Merge dataframes together and export to full .csv
full_b = pd.merge(df, df_b, on="imdbID")
#full_b.to_csv("primary_merge.csv", index=false)

#### Clean Data

In [ ]:
# Stephanie Hartje
# Load and inspect merged datafile
df = pd.read_csv("primary_merge.csv")
df.head()

In [ ]:
# Stephanie Hartje
# Remove unnamed data columns with no usable information
del df['Unnamed: 0_x']
del df['Unnamed: 0_y']
del df['Unnamed: 0.1']

In [ ]:
# Stephanie Hartje 
# Remove columns not needed for analysis, or that indicate the subject is not a movie (tv series etc).
#Note from Steph - del df_movie['year'] looks like it's included twice and it's generating an error message.
#For now I commented out the duplicate.  If others agree it's a duplicate we can delete.
del df_movie['Episode']
del df_movie['Error']
del df_movie['Response']
del df_movie['Season']
del df_movie['Type']
del df_movie['seriesID']
del df_movie['totalSeasons']
del df_movie['title']
del df_movie['year']
del df_movie['imdbid']
del df_movie['id']
#del df_movie['year']

df_movie.head()

In [ ]:
# Stephanie Hartje
#SHartje's Edits start here
#Fixing characters that were messed up when data was pulled, merged, etc.

#Create a list of the column headings
Titles = list(df_movie)

#Cycle through the list of column headings and convert each column from iso-8859-1 to utf-8
for Title in Titles:
    try:
        df_movie[Title] = df_movie[Title].str.decode('iso-8859-1').str.encode('utf-8')
    except:
        df_movie[Title] = df_movie[Title]

#display the dataframe to check 
df_movie.head(30)

In [ ]:
# Stephanie Hartje
# Export clean data to .csv for group to use in analyses
#df_movie.to_csv("clean_data.csv")